In [2]:
import torch
import pandas as pd
import numpy as np
from transformers import MobileViTImageProcessor
import os
from dataset import MRIImageDataModule, get_transform
from models import MobileViTLightning, EfficientNetBaseline
from utils import get_best_device

# Set device
device = get_best_device()

# Path to the CSV file
csv_path = "Data/metadata_for_preprocessed_files.csv"

# Define the models and slices
model_configs = {
    "mobilevit-s": {
        "model_ckpt": "apple/mobilevit-small",
        "slice_numbers": ["65", "86", "56", "95", "62", "35", "59", "74", "80", "134"],
    },
    "efficientnet-b2": {
        "model_ckpt": None,
        "slice_numbers": ["65", "86", "56", "95", "62", "35", "59", "74", "80", "134"],
    },
}


# Function to store feature maps into a dictionary
def extract_features_to_dict(
    model_name, model, data_loader, slice_number, feature_dict
):
    with torch.no_grad():
        for batch in data_loader:
            inputs = batch["inputs"]
            labels = batch["labels"]
            ids = batch["id"]
            inputs = inputs.to(device).float()
            features = model.extract_features(inputs)
            features = features.cpu().numpy()
            for i, id_ in enumerate(ids):
                if id_ not in feature_dict:
                    feature_dict[id_] = {"label": labels[i].item()}
                feature_dict[id_][f"slice_{slice_number}"] = features[i].tolist()


# Function to convert feature dictionary to DataFrame and save as CSV
def save_features_dict_to_csv(feature_dict, file_name):
    df = pd.DataFrame.from_dict(feature_dict, orient="index")
    df = df.reset_index().rename(columns={"index": "id"})
    df.to_csv(file_name, index=False)
    print(f"Saved features to {file_name}")


# Main loop to extract and save features
for stage in ["train", "val", "test"]:
    for model_name, config in model_configs.items():
        feature_dict = {}
        model_ckpt = config["model_ckpt"]
        transform = get_transform(model_name, model_ckpt)

        for slice_number in config["slice_numbers"]:
            model_path = f"model_checkpoints/with_custom_sampler/{model_name}/slice_number_{slice_number}_with_sampling_weights.ckpt"
            if not os.path.exists(model_path):
                continue

            # Load the model
            if model_name == "mobilevit-s":
                model = MobileViTLightning.load_from_checkpoint(
                    model_path, model_ckpt=model_ckpt, num_labels=4
                )
            elif model_name == "efficientnet-b2":
                model = EfficientNetBaseline.load_from_checkpoint(
                    model_path, model_name="efficientnet-b2", num_classes=4
                )

            model = model.to(device)
            model.eval()

            # Initialize the data module
            data_module = MRIImageDataModule(
                csv_path,
                slice_number=int(slice_number),
                transform=transform,
                batch_size=48,
                num_workers=0,
                always_return_id=True,
            )
            data_module.setup()

            # Get data loaders
            if stage == "train":
                data_loader = data_module.train_dataloader()
            elif stage == "val":
                data_loader = data_module.val_dataloader()
            elif stage == "test":
                data_loader = data_module.test_dataloader()

            # Extract and store features
            extract_features_to_dict(
                model_name, model, data_loader, slice_number, feature_dict
            )

        # Save feature dictionary to CSV
        save_features_dict_to_csv(
            feature_dict,
            f"extracted_features/only_custom_weights/{model_name}_{stage}_features.csv",
        )

/Users/henrismidt/anaconda3/envs/alzheimer/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of MobileViTForImageClassification were not initialized from the model checkpoint at apple/mobilevit-small and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 640]) in the checkpoint and torch.Size([4, 640]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/henrismidt/anaconda3/envs/alzheimer/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecat

Saved features to extracted_features/only_custom_weights/mobilevit-s_train_features.csv
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Saved features to extracted_features/only_custom_weights/efficientnet-b2_train_features.csv


/Users/henrismidt/anaconda3/envs/alzheimer/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of MobileViTForImageClassification were not initialized from the model checkpoint at apple/mobilevit-small and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 640]) in the checkpoint and torch.Size([4, 640]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/henrismidt/anaconda3/envs/alzheimer/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecat

Saved features to extracted_features/only_custom_weights/mobilevit-s_val_features.csv
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Saved features to extracted_features/only_custom_weights/efficientnet-b2_val_features.csv


/Users/henrismidt/anaconda3/envs/alzheimer/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of MobileViTForImageClassification were not initialized from the model checkpoint at apple/mobilevit-small and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 640]) in the checkpoint and torch.Size([4, 640]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/henrismidt/anaconda3/envs/alzheimer/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecat

Saved features to extracted_features/only_custom_weights/mobilevit-s_test_features.csv
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b2
Saved features to extracted_features/only_custom_weights/efficientnet-b2_test_features.csv


In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

csv_names = [
    "efficientnet-b2_train_features",
    "efficientnet-b2_val_features",
    "efficientnet-b2_test_features",
    "MobileVit-s_train_features",
    "MobileVit-s_val_features",
    "MobileVit-s_test_features",
]

for csv_name in csv_names:
    # Read the CSV file
    df = pd.read_csv(f"extracted_features/only_custom_weights/{csv_name}.csv")

    # Get slice columns
    slice_cols = [col for col in df.columns if col.startswith("slice_")]

    # Function to apply global average pooling to a single feature map
    def global_average_pooling(feature_map):
        feature_map_array = np.array(eval(feature_map))
        return np.mean(feature_map_array, axis=(1, 2))

    def global_max_pooling(feature_map):
        feature_map_array = np.array(eval(feature_map))
        return np.max(feature_map_array, axis=(1, 2))

    # Apply global average pooling to all slice columns with a progress bar
    for col in tqdm(slice_cols, desc=f"Processing {csv_name}"):
        df[col] = df[col].apply(global_max_pooling)

    # Save the resulting dataframe to a new CSV file
    with open(f"{csv_name}_only_custom_weights_pooled.pkl", "wb") as f:
        pickle.dump(df, f)

    print(
        f"Global average pooling applied and saved to 'extracted_features/{csv_name}_pooled.pkl'"
    )

    for col in df.columns:
        if col.startswith("slice_"):
            shapes = df[col].apply(lambda x: x.shape).unique()
            print(f"Unique shapes in column {col} of {csv_name}: {shapes}")

Processing efficientnet-b2_train_features: 100%|██████████| 10/10 [04:21<00:00, 26.13s/it]


Global average pooling applied and saved to 'extracted_features/efficientnet-b2_train_features_pooled.pkl'
Unique shapes in column slice_65 of efficientnet-b2_train_features: [(1408,)]
Unique shapes in column slice_86 of efficientnet-b2_train_features: [(1408,)]
Unique shapes in column slice_56 of efficientnet-b2_train_features: [(1408,)]
Unique shapes in column slice_95 of efficientnet-b2_train_features: [(1408,)]
Unique shapes in column slice_62 of efficientnet-b2_train_features: [(1408,)]
Unique shapes in column slice_35 of efficientnet-b2_train_features: [(1408,)]
Unique shapes in column slice_59 of efficientnet-b2_train_features: [(1408,)]
Unique shapes in column slice_74 of efficientnet-b2_train_features: [(1408,)]
Unique shapes in column slice_80 of efficientnet-b2_train_features: [(1408,)]
Unique shapes in column slice_134 of efficientnet-b2_train_features: [(1408,)]


Processing efficientnet-b2_val_features: 100%|██████████| 10/10 [00:43<00:00,  4.31s/it]


Global average pooling applied and saved to 'extracted_features/efficientnet-b2_val_features_pooled.pkl'
Unique shapes in column slice_65 of efficientnet-b2_val_features: [(1408,)]
Unique shapes in column slice_86 of efficientnet-b2_val_features: [(1408,)]
Unique shapes in column slice_56 of efficientnet-b2_val_features: [(1408,)]
Unique shapes in column slice_95 of efficientnet-b2_val_features: [(1408,)]
Unique shapes in column slice_62 of efficientnet-b2_val_features: [(1408,)]
Unique shapes in column slice_35 of efficientnet-b2_val_features: [(1408,)]
Unique shapes in column slice_59 of efficientnet-b2_val_features: [(1408,)]
Unique shapes in column slice_74 of efficientnet-b2_val_features: [(1408,)]
Unique shapes in column slice_80 of efficientnet-b2_val_features: [(1408,)]
Unique shapes in column slice_134 of efficientnet-b2_val_features: [(1408,)]


Processing efficientnet-b2_test_features: 100%|██████████| 10/10 [00:44<00:00,  4.42s/it]


...

Global average pooling applied and saved to 'extracted_features/MobileVit-s_train_features_pooled.pkl'
Unique shapes in column slice_65 of MobileVit-s_train_features: [(160,)]
Unique shapes in column slice_86 of MobileVit-s_train_features: [(160,)]
Unique shapes in column slice_56 of MobileVit-s_train_features: [(160,)]
Unique shapes in column slice_95 of MobileVit-s_train_features: [(160,)]
Unique shapes in column slice_62 of MobileVit-s_train_features: [(160,)]
Unique shapes in column slice_35 of MobileVit-s_train_features: [(160,)]
Unique shapes in column slice_59 of MobileVit-s_train_features: [(160,)]
Unique shapes in column slice_74 of MobileVit-s_train_features: [(160,)]
Unique shapes in column slice_80 of MobileVit-s_train_features: [(160,)]
Unique shapes in column slice_134 of MobileVit-s_train_features: [(160,)]


Processing MobileVit-s_val_features: 100%|██████████| 10/10 [00:05<00:00,  1.85it/s]


Global average pooling applied and saved to 'extracted_features/MobileVit-s_val_features_pooled.pkl'
Unique shapes in column slice_65 of MobileVit-s_val_features: [(160,)]
Unique shapes in column slice_86 of MobileVit-s_val_features: [(160,)]
Unique shapes in column slice_56 of MobileVit-s_val_features: [(160,)]
Unique shapes in column slice_95 of MobileVit-s_val_features: [(160,)]
Unique shapes in column slice_62 of MobileVit-s_val_features: [(160,)]
Unique shapes in column slice_35 of MobileVit-s_val_features: [(160,)]
Unique shapes in column slice_59 of MobileVit-s_val_features: [(160,)]
Unique shapes in column slice_74 of MobileVit-s_val_features: [(160,)]
Unique shapes in column slice_80 of MobileVit-s_val_features: [(160,)]
Unique shapes in column slice_134 of MobileVit-s_val_features: [(160,)]


Processing MobileVit-s_test_features: 100%|██████████| 10/10 [00:05<00:00,  1.82it/s]

Global average pooling applied and saved to 'extracted_features/MobileVit-s_test_features_pooled.pkl'
Unique shapes in column slice_65 of MobileVit-s_test_features: [(160,)]
Unique shapes in column slice_86 of MobileVit-s_test_features: [(160,)]
Unique shapes in column slice_56 of MobileVit-s_test_features: [(160,)]
Unique shapes in column slice_95 of MobileVit-s_test_features: [(160,)]
Unique shapes in column slice_62 of MobileVit-s_test_features: [(160,)]
Unique shapes in column slice_35 of MobileVit-s_test_features: [(160,)]
Unique shapes in column slice_59 of MobileVit-s_test_features: [(160,)]
Unique shapes in column slice_74 of MobileVit-s_test_features: [(160,)]
Unique shapes in column slice_80 of MobileVit-s_test_features: [(160,)]
Unique shapes in column slice_134 of MobileVit-s_test_features: [(160,)]
